In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_train = pd.read_csv('/content/drive/MyDrive/Hackaton/df_total_train.csv', sep=',', index_col=0)

In [ ]:
data_val = pd.read_csv('/content/drive/MyDrive/Hackaton/df_total_val.csv', sep=',', index_col=0)

In [ ]:
data_test = pd.read_csv('/content/drive/MyDrive/Hackaton/df_total_test.csv', sep=',', index_col=0)

In [ ]:
data_train_1 = pd.read_csv('/content/drive/MyDrive/Hackaton/train.csv', sep=',', index_col=0)

In [ ]:
data_val_1 = pd.read_csv('/content/drive/MyDrive/Hackaton/val.csv', sep=',', index_col=0)

In [ ]:
data_test_1 = pd.read_csv('/content/drive/MyDrive/Hackaton/test.csv', sep=',', index_col=0)

In [ ]:
data_train['user-id'] = data_train.index
data_train_1 = data_train_1.drop(['target'], axis=1)
df_train = data_train.merge(data_train_1, how='left', on='user-id')
df_train = df_train.drop(['user-id'], axis=1)

In [ ]:
data_val['user-id'] = data_val.index
data_val_1 = data_val_1.drop(['target'], axis=1)
df_val = data_val.merge(data_val_1, how='left', on='user-id')
df_val = df_val.drop(['user-id'], axis=1)

In [ ]:
data_test['user-id'] = data_test.index
df_test = data_test.merge(data_test_1, how='left', on='user-id')
df_test = df_test.drop(['user-id'], axis=1)

In [ ]:
df_test.to_csv('/content/drive/MyDrive/Hackaton/df_test.csv')

In [ ]:
df_val.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 27447 entries, 0 to 27446
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   target                             27447 non-null  int64  
 1   es_female                          27447 non-null  int64  
 2   es_male                            27447 non-null  int64  
 3   es_neutrale                        27447 non-null  int64  
 4   last_visit_other                   27447 non-null  int64  
 5   last_visit_electronics             27447 non-null  int64  
 6   last_visit_hypermarket             27447 non-null  int64  
 7   last_visit_bank                    27447 non-null  int64  
 8   last_visit_entertainment_services  27447 non-null  int64  
 9   last_visit_fashion                 27447 non-null  int64  
 10  last_visit_media                   27447 non-null  int64  
 11  last_visit_cosmetics               27447 non-null  int

Обучение модели

In [ ]:
df_val = df_val.dropna()

In [ ]:
df_train = df_train.dropna()

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
X_train = df_train.drop('target', axis=1)
y_train = df_train.target

In [ ]:
X_val = df_val.drop('target', axis=1)
y_val = df_val.target

In [ ]:
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic'
    , n_estimators=250
    , learning_rate=0.10
    , colsample_bytree=0.70
    , max_depth=3
    , n_jobs=4
)

In [ ]:
xgb_model.fit(
    X_train, y_train
    , early_stopping_rounds=5
    , eval_set=[(X_test, y_test)]
    , verbose=False
)

print('Training Done!')

predictions = xgb_model.predict(X_val)

print('Predictions Done!')

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Training Done!
Predictions Done!


In [ ]:
accuracy = accuracy_score(y_val, predictions)
train_score = round(xgb_model.score(X_train, y_train) * 100, 2)
valid_score = round(xgb_model.score(X_val, y_val) * 100, 2)

print('Accuracy: ', round(accuracy * 100, 2), '%', sep='')

Accuracy: 80.55%


predict

In [ ]:
y_pred = xgb_model.predict(df_test)